# Blocking and Matching Process of Project Stage 3
####     Authors: Jiewei Hong, Yusong Yang, Wei Tang
This Jupyter notebook explains a basic workflow of processing entity matching between two tables using py_entitymatching. The dataset we are using is about the information of restaurants: the two tables are restaurant information originally from Yelp and Zomato, which include the restaurant names, addresses, zipcode, phone numbers, cuisine, rate/vote and reviews on each website. Our goal is to find the entities from Yelp table and Zomato table that are refering to the same real-world restaurant. The dataset can be found via the link: https://sites.google.com/site/anhaidgroup/useful-stuff/data.

Entity matching between two tables typically consists of the following three steps:
    1. Reading the input tables (and do data cleaning or other pre-processing if necessary);
    2. Blocking the input tables to get a candidate set;
    3. Matching the tuple pairs in the candidate set.

And the first thing is to import the package py_entitymatching and other libraries as follows:


In [1]:
import sys
sys.path.append('/home/hjw/anaconda2/lib/python2.7/site-packages/py_entitymatching/')

import py_entitymatching as em
import pandas as pd
import numpy as np
from sklearn import metrics

In [2]:
# Display the versions
print('python version: ' + sys.version)
print('pandas version: ' + pd.__version__)
print('magellan version: ' + em.__version__)

python version: 2.7.13 |Anaconda custom (64-bit)| (default, Dec 20 2016, 23:09:15) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
pandas version: 0.19.2
magellan version: 0.1.0



## 1. Read Input Tables
Before actually read in the tables, we begin by cleaning the original table. We notice that the original tables are not very clean: there are some restaurant names and addresses with non-ascii characters. So we substitute all non-ascii characters in address and name attributes by space for both two tables. 
Also, some abbreviations are used in the tables, such as Ave. for Avenue, Blvd. for Boulevard. We perform substition on those unabbreviated words (for example, substitute Avenue by Ave.).
After data cleaning, we read in the two tables, name the Yelp table as AA, name the Zomato table as BB.

In [3]:
#load data
AA = em.read_csv_metadata('/home/hjw/cs838/stage3/restaurant/restaurants2/final/AA.csv',key='ID') # Yelp Table
BB = em.read_csv_metadata('/home/hjw/cs838/stage3/restaurant/restaurants2/final/BB.csv',key='ID') # Zomato Table

No handlers could be found for logger "py_entitymatching.io.parsers"


In [4]:
print('Number of tuples in A: ' + str(len(AA)))
print('Number of tuples in B: ' + str(len(AA)))
print('Number of tuples in A X B (i.e., the Cartesian product): ' + str(len(AA)*len(BB)))

Number of tuples in A: 3897
Number of tuples in B: 3897
Number of tuples in A X B (i.e., the Cartesian product): 27123120


In [5]:
AA.head()

,ID,name,votes,rating,phone,address,zip,cuisine
0,0,Patino s Grill,35,5.0,(773) 280-9562,2943 W Irving Park Rd,60618,American (Traditional)
1,1,Grandma J s Local Kitchen,188,4.0,(773) 227-3626,1552 N Kedzie Ave,60651,Breakfast & Brunch
2,2,Ada St,340,4.5,(773) 697-7069,1664 N Ada St,60642,Bars
3,3,Fuh,104,4.5,(773) 270-4418,2218 N Lincoln Ave,60614,Vietnamese
4,4,Staropolska Restaurant,286,4.5,(773) 342-0779,3030 N Milwaukee Ave,60618,Polish


In [6]:
BB.head()

,ID,name,votes,rating,phone,address,zip,cuisine,reviewcount
0,0,Strings Ramen Shop,15,3.1,(312) 374-3450,"2141 S. Archer Ave, Chicago",60616,"Asian,Chinese,Ramen",2
1,1,Francesco's Hole In The Wall,179,4.0,(847) 272-0155,254 Skokie Blvd,60062,Italian,6
2,3,Four Belly,12,3.0,(773) 661-6182,"3227 N. Clark St, Chicago",60657,"Asian,BBQ,Japanese",0
3,4,Max's Take Out,60,4.6,(312) 553-0170,"20 E. Adams St, Chicago",60603,"American,Burger",12
4,5,BluFish Sushi Bistro,150,4.0,(847) 272-7990,"Plaza Del Prado, 2733 Pfingsten Rd",60026,"Sushi,Japanese",10


In [7]:
# Display the keys of the input tables
em.get_key(AA), em.get_key(BB)

('ID', 'ID')

## 2. Downsampling(Skipped)
Our dataset is not too large, so we skip the downsampling step and go to the blocking process.


## 3. Blocking Process
### Block Tables to Generate Candidate Set
Before we do the matching, we would like to remove the obviously non-matching tuple pairs from the input tables. This would reduce the number of tuple pairs considered in the matching process.  
For this entity matching project, we know that if two tuples from two tables are refering to the same restaurant in real world, they much have similar address. So we first do overlap blocking on address to get a table C1. Then, if two entity are refering to the same restaurant, the name should at least have some overlap, thus we do overlap blocking based on name on C1, setting q-gram's q value as 3, generate C2.  
The reason we do blocking on address before name is, it's possible that two restaurants are the chain-restaurants,i.e., they have identical name but distinct address. 

In [8]:
# Create an Overlap blocker
ob = em.OverlapBlocker()
# block on address
C1 = ob.block_tables(AA, BB, 'address', 'address', overlap_size = 2,l_output_attrs=list(AA), r_output_attrs=list(BB))
len(C1)

0%                          100%
[##############################] | ETA: 00:00:10 | ETA: 00:00:10 | ETA: 00:00:09 | ETA: 00:00:08 | ETA: 00:00:07 | ETA: 00:00:06 | ETA: 00:00:06 | ETA: 00:00:05 | ETA: 00:00:05 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:05


573192

In [9]:
# block on name
C2 = ob.block_candset(C1, 'name', 'name', word_level=False, q_val=3, overlap_size = 2)
len(C2)

0%                          100%
[##############################] | ETA: 00:00:16 | ETA: 00:00:16 | ETA: 00:00:15 | ETA: 00:00:14 | ETA: 00:00:13 | ETA: 00:00:13 | ETA: 00:00:12 | ETA: 00:00:12 | ETA: 00:00:11 | ETA: 00:00:10 | ETA: 00:00:10 | ETA: 00:00:09 | ETA: 00:00:09 | ETA: 00:00:08 | ETA: 00:00:08 | ETA: 00:00:07 | ETA: 00:00:07 | ETA: 00:00:06 | ETA: 00:00:06 | ETA: 00:00:05 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:16


38958

Next, we perform a blackbox blocker on C2, which conducts a blocking based on the attribute "zip". Since we notice that, there exists many pairs that don't have exactly the same zip code but actually refer to the same real-world restaurants (could be resulted from some zip code changing policies), and realizing that C2 is already a subset that should have similar address, our rule is not very strict for 'zip': if the first two digit of zipcode are not the same for two tuples, then they are blocked.

In [10]:
# Define a Black-box rule based on five-digits zip code: if the first two digit of zipcode are not the same 
# for two tuples, then they are blocked
def match_by_zip(ltuple, rtuple):
    l_km = ltuple['zip']
    r_km = rtuple['zip']
    if abs(l_km - r_km) > 1000.0 :
        return True
    else:
        return False

In [11]:
# Create a Black-box blocker and perform blocking
bb = em.BlackBoxBlocker()
bb.set_black_box_function(match_by_zip)
C3 = bb.block_candset(C2)
len(C3)

0%                          100%
[##############################] | ETA: 00:00:04 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:02


18820

By this step, the candidate set (C3) has been downsized to 18820. The fourth blocker is an blackbox blocker that performs blocking on C3 based on the first three digits of telephone number. If one pair of tuples doesn't share the same first three digits of phone number (which represents area), they are blocked.

In [12]:
# Define a Black-box rule based on first three digits of phone number: If one pair of tuples doesn't share the same 
# first three digits of phone number (which represents area), they are blocked.
def match_first_three_tele_num (ltuple, rtuple):
	l_area_code = ltuple['phone'].split()[0]
	r_area_code = rtuple['phone'].split()[0]
	if (l_area_code == r_area_code):
		return False
	else:
		return True

In [13]:
# Create a Black-box blocker and perform blocking
bb = em.BlackBoxBlocker()
bb.set_black_box_function(match_first_three_tele_num)
C4 = bb.block_candset(C3)
len(C4)

0%                          100%
[##############################] | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:01


11765

The size of candidate set right now (C4) is 11765. Next, we switch to debug on the blocker output.

### Debug on Blocker Output
We want to make sure that out blockers didn't drop to many potential metches. We exam this by looking at the first ten pairs:

In [14]:
#Debug the blockers
D = em.debug_blocker(C2,AA,BB)
em.to_csv_metadata(D,'/home/hjw/cs838/stage3/restaurant/restaurants2/final/debug_block.csv')
D.head(10)

,_id,similarity,ltable_ID,rtable_ID,ltable_name,ltable_phone,ltable_address,ltable_cuisine,rtable_name,rtable_phone,rtable_address,rtable_cuisine
0,0,0.600000,3049,5707,Mr. Brews Taphouse,(608) 845-2280,611 Hometown CirSte 104,American (Traditional),Mr. Brews Taphouse,(608) 845-2280,Hometown Circle,"Burger,Bar Food"
1,1,0.555556,2991,5500,Athens Gyros,(608) 249-6720,5420 Willow Rd,Greek,Trader Gus,(608) 249-6720,5420 willow Rd,Greek
2,2,0.545455,1976,2681,Hatchet Hall,(310) 391-4222,12517 W Washington Blvd,American (Traditional),Waterloo & City,(310) 391-4222,12517 W Washington Blvd,NaN
3,3,0.461538,3160,5453,Sarku Japan,(608) 836-0818,108 W Towne Mall,Japanese,Kato's Cajun,(608) 836-0818,"108 W Towne Mall Ste 157, Madison",Asian
4,4,0.454545,1947,4183,Ciabatta Bar,(323) 500-1121,6464 W Sunset Blvd,Italian,Bar Marmont,(323) 650-0575,8171 W Sunset Blvd,"American,European"
5,5,0.454545,1244,1914,Saint Ann,(214) 782-9807,2501 N Harwood St,American (New),Mercat Bistro,(214) 953-0917,2501 N Harwood St,French
6,6,0.454545,2379,4994,The Paradise Lounge,(608) 256-2263,119 W Main St,Dive Bars,Borland's,(608) 424-3013,119 W Main St,American
7,7,0.454545,1516,1666,Meddlesome Moth,(214) 628-7900,1621 Oak Lawn Ave,Bars,Ascension Coffee,(214) 741-3211,1621 Oak Lawn Ave,"Cafe,American"
8,8,0.454545,2847,5472,Pooley s,(608) 242-1888,5441 High Crossing Blvd,Venues & Event Spaces,Diggity's,(608) 242-1888,"5441 High Crossing Blvd, Madison",Sandwich
9,9,0.437500,770,2265,Lake House Bar and Grill,(214) 484-8624,7510 E NW Hwy,American (New),Lake House Bar & Grill,(214) 484-8624,"7510 East Northwest Highway, Dallas","American,Burger,Seafood"


In those ten pairs, only two (the first and last) pairs are incorrectly blocked. By checking back the original tables, we find out the causes for them: the first one is a result of misplet address in Zomato table, the last one is a result of heavily abbreviated address in Yelp table. Since the table is sorted by simliarity score (from high to low), we conclude that such percentage of incorrectness is accepctable.  
Since we think that 10K of tuples are a reasonable size, we decide to stop blocking here. Before we start the matching process, we save the final version of candidate set after the last blocker (C4) into a csv file.

In [15]:
em.to_csv_metadata(C4,'./C4.csv')

True

## Matching the Tuple Pairs in the Candidate Set
In the matching process, we would want to match the tuple pairs in the candidate set. Specifically, we use learning-based method for matching purposes. This typically involves the following four step:
    1. Sampling and labeling the candidate set;
    2. Splitting the labled data into development set and evaluation set;
    3. Selecting the best learning based matcher using the development set;
    3. Evaluating the selected matcher using the evaluation set.

## 4. Sampling and Labeling the Candidate Set
First, we randomly sample 500 pairs and save for labeling purpose. 

In [16]:
Sample = em.sample_table(C4,sample_size=500)
em.to_csv_metadata(Sample,'./labeled_data.csv')

True

The data are then manually labelled. The labelled data is file 'labeled_data_1.csv'. The irrelevant attribute in labeled data (e.g., votes, rating) are deleted.
The corresponding attributes in Tables AA and BB are also deleted. Then we read the labeled data and call it G:

In [17]:
# copy tables AA and BB and then delete irrelevant attributes
AA1 = AA
BB1 = BB
del AA1['votes']
del AA1['rating']
del AA1['cuisine']
del BB1['votes']
del BB1['rating']
del BB1['cuisine']
del BB1['reviewcount']

# Display labeled pairs
G = em.read_csv_metadata('/home/hjw/cs838/stage3/restaurant/restaurants2/final/labeled_data_1.csv', key='_id',ltable=AA1,rtable=BB1,fk_ltable='ltable_ID', fk_rtable='rtable_ID')
len(G)
G.head()

,_id,ltable_ID,rtable_ID,ltable_name,ltable_phone,ltable_address,ltable_zip,rtable_name,rtable_phone,rtable_address,rtable_zip,gold
0,723,279,7,Gyros On The Spit,(773) 698-7999,2826 N Lincoln Ave,60657,Father & Son,(773) 252-2620,"2475 N. Milwaukee Ave, Chicago",60647,0
1,738,363,7,The Garage Bar & Sandwiches,(773) 647-1386,6154 N Milwaukee Ave,60646,Father & Son,(773) 252-2620,"2475 N. Milwaukee Ave, Chicago",60647,0
2,8222,256,50,Takos Koreanos,(773) 654-1220,1706 W Foster Ave,60640,L' Patron Tacos,(773) 252-6335,"2815 W. Diversey Ave, Chicago",60647,0
3,8312,689,50,Los 3 Panchos,(773) 935-3198,1155 W Diversey Pkwy,60614,L' Patron Tacos,(773) 252-6335,"2815 W. Diversey Ave, Chicago",60647,0
4,9221,352,58,Groundswell Coffee Roasters,(773) 754-7590,4839 N Damen Ave,60625,Northside Bar and Grill,(773) 384-3555,"1635 N. Damen Ave, Chicago",60647,0


## 5. Splitting the Labled Data into Development Set and Evaluation Set
In this step, we split the labeled data into two sets: development (I) and evaluation (J). Specifically, the development set is used to come up with the best learning-based matcher, and evaluation set used to evaluate the selected matcher with only one time.

In [18]:
##divide the train set and test set, whcih is set I (devel_set) and J (eval_set) repectively
train_test = em.split_train_test(G, train_proportion=0.6)
#set the development set
devel_set = train_test['train']
#set the test set
eval_set = train_test['test']
#save train set and test set
em.to_csv_metadata(devel_set,'./SetI.csv')
em.to_csv_metadata(eval_set,'./SetJ.csv')

True

## 6. Creating a set of learning-based matchers

In [19]:
#construct six learning-based matcher 
dt = em.DTMatcher(name='DT', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NB')

## 7. Selecting the Best Learning-Based Matcher Using the Development Set


### a. Creating features table
Since we delete some of the irrelevant attribute in labeled data G, we don't want to generate features related to these attributes (namely votes, rating, cuisine and reviewcount'), so we first delete those features from original tables AA and BB, and then generate feature tables.

In [20]:
#construct feature table
feature_table = em.get_features_for_matching(AA1,BB1)

### b. Convering I into a set H of feature vectors 

In [21]:
#construct feature vector for each each candidate pair
H = em.extract_feature_vecs(devel_set,feature_table=feature_table, attrs_after='gold',show_progress=False)

### c. Filling in the missing values if any (skipped)
Our dataset doesn't have missing value issue, so we just skip this step.

### d. Selecting the best learning-based matcher using k-fold cross validation
There is still some features in H that we want to exclude, we put these features in to a cell attr_to_exclude, and set the exclude_attrs to be it when we do cross validation. To guarantee each time we generate the same fold for obtain f1, precision and recall, we set random_state=0.

In [22]:
#do the cross validation
attr_to_exclude=['_id', 'ltable_ID', 'rtable_ID', 'gold'];
result_f1 = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, exclude_attrs=attr_to_exclude, k=5, target_attr='gold', metric='f1', random_state=0)   
result_prec = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, exclude_attrs=attr_to_exclude, k=5, target_attr='gold', metric='precision', random_state=0)   
result_rec = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, exclude_attrs=attr_to_exclude, k=5, target_attr='gold', metric='recall', random_state=0)   

In [23]:
#show f1/precision/recall on each ML model
result_f1['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DT,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x7f4e9f6da190>,5,0.956522,1.0,0.947368,0.875,0.818182,0.919414
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x7f4e9f6da110>,5,1.000000,1.0,1.000000,0.875,0.962963,0.967593
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x7f4e9f6da9d0>,5,0.666667,1.0,0.888889,0.800,0.700000,0.811111
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x7f4e9f6da650>,5,1.000000,1.0,1.000000,0.875,0.962963,0.967593
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x7f4e9f6da350>,5,1.000000,1.0,1.000000,0.875,0.880000,0.951000
5,NB,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x7f4e9f6dad50>,5,1.000000,1.0,1.000000,0.875,0.962963,0.967593


In [24]:
result_prec['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DT,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x7f4e9f6da190>,5,1.0,1.0,1.0,0.777778,1.000000,0.955556
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x7f4e9f6da110>,5,1.0,1.0,1.0,0.777778,0.928571,0.941270
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x7f4e9f6da9d0>,5,1.0,1.0,1.0,0.750000,1.000000,0.950000
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x7f4e9f6da650>,5,1.0,1.0,1.0,0.777778,0.928571,0.941270
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x7f4e9f6da350>,5,1.0,1.0,1.0,0.777778,0.916667,0.938889
5,NB,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x7f4e9f6dad50>,5,1.0,1.0,1.0,0.777778,0.928571,0.941270


In [25]:
result_rec['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DT,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x7f4e9f6da190>,5,0.916667,1.0,0.9,1.000000,0.692308,0.901795
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x7f4e9f6da110>,5,1.000000,1.0,1.0,1.000000,1.000000,1.000000
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x7f4e9f6da9d0>,5,0.500000,1.0,0.8,0.857143,0.538462,0.739121
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x7f4e9f6da650>,5,1.000000,1.0,1.0,1.000000,1.000000,1.000000
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x7f4e9f6da350>,5,1.000000,1.0,1.0,1.000000,0.846154,0.969231
5,NB,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x7f4e9f6dad50>,5,1.000000,1.0,1.0,1.000000,1.000000,1.000000


We find that all matchers are fairly good. After Considering both precision and recall, we select Naive Bayes matcher.

### e. Debugging the matcher
Though we already have high precision and recall, we still want to see whether we can improve our matcher. Thus we combine the two most important attribute, name and address, calculate the jaccard score of the combination of this two, we add this feature to the feature_table as a new feature.

In [26]:
#Debug ML model, add a new feature, which combine the name and address and calculate the jaccard score
sim = em.get_sim_funs_for_matching()
tok = em.get_tokenizers_for_matching()

feature_string = """jaccard(wspace((ltuple['name'] + ' ' + ltuple['address']).lower()), 
                            wspace((rtuple['name'] + ' ' + rtuple['address']).lower()))"""
feature = em.get_feature_fn(feature_string, sim, tok)

em.add_feature(feature_table, 'jac_ws_name_address', feature)

H = em.extract_feature_vecs(devel_set,feature_table=feature_table, attrs_after='gold',show_progress=False)

Next, test the perfomance of all ML models after adding the new feature.

In [27]:
#Test the perfomance of all ML model after adding the new feature
resultf_f1 = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, exclude_attrs=attr_to_exclude, k=5, target_attr='gold', metric='f1',random_state=0)
resultf_prec = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, exclude_attrs=attr_to_exclude, k=5, target_attr='gold', metric='precision',random_state=0) 
resultf_rec = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, exclude_attrs=attr_to_exclude, k=5, target_attr='gold', metric='recall',random_state=0)

In [28]:
# Display f1/precision/recall scores
resultf_f1['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DT,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x7f4e9f6da190>,5,0.956522,1.0,0.947368,0.875,0.833333,0.922445
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x7f4e9f6da110>,5,0.956522,1.0,1.000000,0.875,0.923077,0.950920
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x7f4e9f6da9d0>,5,0.666667,1.0,0.888889,0.800,0.700000,0.811111
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x7f4e9f6da650>,5,1.000000,1.0,1.000000,0.875,0.962963,0.967593
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x7f4e9f6da350>,5,1.000000,1.0,1.000000,0.875,0.880000,0.951000
5,NB,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x7f4e9f6dad50>,5,1.000000,1.0,1.000000,0.875,0.962963,0.967593


In [29]:
resultf_prec['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DT,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x7f4e9f6da190>,5,1.0,1.0,1.0,0.777778,0.909091,0.937374
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x7f4e9f6da110>,5,1.0,1.0,1.0,0.777778,0.923077,0.940171
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x7f4e9f6da9d0>,5,1.0,1.0,1.0,0.750000,1.000000,0.950000
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x7f4e9f6da650>,5,1.0,1.0,1.0,0.777778,0.928571,0.941270
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x7f4e9f6da350>,5,1.0,1.0,1.0,0.777778,0.916667,0.938889
5,NB,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x7f4e9f6dad50>,5,1.0,1.0,1.0,0.777778,0.928571,0.941270


In [30]:
resultf_rec['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DT,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x7f4e9f6da190>,5,0.916667,1.0,0.9,1.000000,0.769231,0.917179
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x7f4e9f6da110>,5,0.916667,1.0,1.0,1.000000,0.923077,0.967949
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x7f4e9f6da9d0>,5,0.500000,1.0,0.8,0.857143,0.538462,0.739121
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x7f4e9f6da650>,5,1.000000,1.0,1.0,1.000000,1.000000,1.000000
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x7f4e9f6da350>,5,1.000000,1.0,1.0,1.000000,0.846154,0.969231
5,NB,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x7f4e9f6dad50>,5,1.000000,1.0,1.0,1.000000,1.000000,1.000000


By comparing the new scores with the old one, we don's see much improvements (since the old one is already good enough). So in the evaluation step, we switch back to the original feature vectors at the first time.

## 8. Evaluating the Best Matcher Y using J
### a. Convering J into a set L of feature vectors
As said before, we still used the features generated automatically, and the ML method is Naive Bayes. Here we first need to regenerate the feature table as well as feature vectors for both set I (devel_set) and set J (eval_set).

In [31]:
#Use the model that generated automatically (without adding new feature)
feature_table = em.get_features_for_matching(AA1,BB1)
H = em.extract_feature_vecs(devel_set,feature_table=feature_table, attrs_after='gold',show_progress=False)
L = em.extract_feature_vecs(eval_set,feature_table=feature_table, attrs_after='gold',show_progress=False)

### b. Filling in the missing values if any (skipped)
Our dataset doesn't have missing value issue, so we just skip this step.

### c. Training the best matcher Y using I

In [32]:
##Train the six matchers on set I  (features for I is called H)
dt.fit(table = H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold'], target_attr='gold')
rf.fit(table = H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold'], target_attr='gold')
svm.fit(table = H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold'], target_attr='gold')
lg.fit(table = H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold'], target_attr='gold')
ln.fit(table = H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold'], target_attr='gold')
nb.fit(table = H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold'], target_attr='gold')

### d. Computing the accuracy of Y
We used the metrics module in skrlearn package for convenience.

In [33]:
label = L['gold'].as_matrix()
##Test the matchers on test set J (features for J is called L)
print('************************DT result******************************')
dt_result = dt.predict(table = L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold'], target_attr='predict')
print(metrics.classification_report(label, dt_result))

print('************************RF resutl******************************')
rf_result = rf.predict(table = L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold'], target_attr='predict')
print(metrics.classification_report(label, rf_result))

print('************************SVM resutl******************************')
svm_result = rf.predict(table = L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold'], target_attr='predict')
print(metrics.classification_report(label, svm_result))

print('************************LogReg resutl******************************')
lg_result = lg.predict(table = L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold'], target_attr='predict')
print(metrics.classification_report(label, lg_result))

print('************************LinReg resutl******************************')
ln_result = ln.predict(table =L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold'], target_attr='predict')
print(metrics.classification_report(label, ln_result))

print('************************NB resutl******************************')
nb_result = nb.predict(table = L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'gold'], target_attr='predict')
print(metrics.classification_report(label, nb_result))


************************DT result******************************
             precision    recall  f1-score   support

          0       0.98      1.00      0.99       170
          1       1.00      0.87      0.93        30

avg / total       0.98      0.98      0.98       200

************************RF resutl******************************
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       170
          1       1.00      1.00      1.00        30

avg / total       1.00      1.00      1.00       200

************************SVM resutl******************************
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       170
          1       1.00      1.00      1.00        30

avg / total       1.00      1.00      1.00       200

************************LogReg resutl******************************
             precision    recall  f1-score   support

          0       0.99      0.99      0.